In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Question 1

In [2]:
def generate_data(n, mu_0, mu_1, sigma, phi):
    
    n_test = n / 5
    n_test = int(n_test)
    
    X_train = np.empty((n, d))
    y_train = np.random.binomial(1, phi, (n, 1))
    for i in range(len(y_train)):
        if y_train[i] == 1:
            X_train[i] = np.random.multivariate_normal(mu_1, sigma, 1)
        else:
            X_train[i] = np.random.multivariate_normal(mu_0, sigma, 1)
    
    X_test = np.empty((n_test, d))
    y_test = np.random.binomial(1, phi, (n_test, 1))
    for i in range(len(y_test)):
        if y_test[i] == 1:
            X_test[i] = np.random.multivariate_normal(mu_1, sigma, 1)
        else:
            X_test[i] = np.random.multivariate_normal(mu_0, sigma, 1)
    
    return X_train, y_train, X_test, y_test

In [3]:
n=20
d = 100
phi=0.8
mu_0 = np.arange(d,0,-1)
mu_1 = -1* np.arange(d,0,-1)
sigma = 2*np.eye(d)
X_train, y_train, X_test, y_test = generate_data(n, mu_0, mu_1, sigma, phi)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((20, 100), (20, 1), (4, 100), (4, 1))

# Question 2

In [4]:
def gradient_descent_log(X_train, y_train, alpha, max_iter):
    
    theta_hat = np.ones((X_train.shape[1], 1))
    for i in range(1000):
        for j in range(X_train.shape[0]):
            X = X_train[j:j+1, :]
            temp = 1. / (1. + np.exp(-(np.dot(X, theta_hat))))
            error = ((temp - y_train[j]) * X)
        theta_hat -= 1e-2 * error.reshape(-1, 1)
        
    return theta_hat

In [5]:
def GDA_estimator(X_train, y_train, d, class_1, class_2):
    
    phi = np.mean(y_train)
    mu_0 = np.sum((y_train==class_2).reshape(-1, 1)*X_train, axis=0) / np.sum(y_train==class_2)
    mu_1 = np.sum((y_train==class_1).reshape(-1, 1)*X_train, axis=0) / np.sum(y_train==class_1)
    sigma_kk = np.mean(np.concatenate((np.square(X_train[(y_train==class_2).ravel()]-mu_0.reshape(1, -1)), np.square(X_train[(y_train==class_1).ravel()]-mu_1.reshape(1, -1))), axis=0), axis=0)
    sigma = np.eye(d)
    for i in range(d):
        sigma[i][i] = sigma_kk[i]
    
    return phi, mu_0, mu_1, sigma

In [6]:
theta_hat_gd = gradient_descent_log(X_train, y_train, 1e-10, 1000)
theta_hat_gd.shape

C:\Users\sbhatta\AppData\Local\Temp/ipykernel_16360/4000100553.py:7: RuntimeWarning: overflow encountered in exp
  temp = 1. / (1. + np.exp(-(np.dot(X, theta_hat))))


(100, 1)

In [7]:
phi, mu_0, mu_1, sigma = GDA_estimator(X_train, y_train, d, 1, 0)
phi, mu_0.shape, mu_1.shape, sigma.shape

(0.8, (100,), (100,), (100, 100))

# Question 3

In [8]:
def log_reg_output(X_test, y_test, theta_hat, class_1, class_2):
    
    h_hat = 1. / (1. + np.exp(-(np.dot(X_test, theta_hat))))
    
    y_LR = np.empty(y_test.shape).reshape(-1, 1)
    y_LR[h_hat > (1-h_hat)] = class_1
    y_LR[h_hat <= (1-h_hat)] = class_2
    
    return y_LR

In [9]:
def GDA_output(X_test, y_test, d, mu_1, mu_0, sigma, phi, class1, class2):
    
    y_GDA = []
    
    for data in X_test:
        y_GDA_1 = 1. / (np.power(np.power(2 * np.pi, d) * np.abs(np.linalg.det(sigma + np.eye(d) * 0.001)), 0.5)) * np.exp(-0.5 * (data - mu_1).reshape(1, -1).dot(np.linalg.inv(sigma + np.eye(d) * 0.001)).dot((data - mu_1).reshape(1, -1).T)) * phi
        y_GDA_0 = 1. / (np.power(np.power(2 * np.pi, d) * np.abs(np.linalg.det(sigma + np.eye(d) * 0.001)), 0.5)) * np.exp(-0.5 * (data - mu_0).reshape(1, -1).dot(np.linalg.inv(sigma + np.eye(d) * 0.001)).dot((data - mu_0).reshape(1, -1).T)) * (1-phi)

        if y_GDA_1 >= y_GDA_0:
            y_GDA.append(class1)
        else:
            y_GDA.append(class2)
    
    return np.array(y_GDA).reshape(-1, 1)

In [10]:
def evaluate_error(y_test, y_hat):
    
    error = np.mean(np.abs(y_test-y_hat))
    
    return error

In [11]:
y_LR = log_reg_output(X_test, y_test, theta_hat_gd, 1, 0)
y_LR.shape

C:\Users\sbhatta\AppData\Local\Temp/ipykernel_16360/3361233284.py:3: RuntimeWarning: overflow encountered in exp
  h_hat = 1. / (1. + np.exp(-(np.dot(X_test, theta_hat))))


(4, 1)

In [12]:
LR_error = evaluate_error(y_test, y_LR)
LR_error

0.0

In [13]:
y_GDA = GDA_output(X_test, y_test, d, mu_1, mu_0, sigma, phi, 1, 0)
y_GDA.shape

(4, 1)

In [14]:
GDA_error = evaluate_error(y_test, y_GDA)
GDA_error

0.0

# Question 4

In [15]:
mnist = fetch_openml('mnist_784')
X = mnist['data']
y = mnist['target']
for i in range(len(y)):
    y[i] = int(y[i])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7)
print(X_train.shape, X_train[y_train==9].shape, X_train[y_train==0].shape)

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

In [16]:
X_train_0 = X_train[y_train==0, :]
y_train_0 = y_train[y_train==0]
X_train_9 = X_train[y_train==9, :]
y_train_9 = y_train[y_train==9]
X_train = np.concatenate((X_train_0, X_train_9), axis=0)
y_train = np.concatenate((y_train_0, y_train_9), axis=0)
X_train.shape, y_train.shape

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
X_test_0 = X_test[y_test==0, :]
y_test_0 = y_test[y_test==0]
X_test_9 = X_test[y_test==9, :]
y_test_9 = y_test[y_test==9]
X_test = np.concatenate((X_test_0, X_test_9), axis=0)
y_test = np.concatenate((y_test_0, y_test_9), axis=0)
X_test.shape,  y_test.shape

In [ ]:
d = X_train.shape[1]

In [ ]:
theta_hat_gd = gradient_descent_log(X_train, y_train, 1e-10, 1000)
theta_hat_gd.shape

In [ ]:
phi, mu_0, mu_1, sigma = GDA_estimator(X_train, y_train, d, 9, 0)
phi, mu_0.shape, mu_1.shape, sigma.shape

In [ ]:
y_LR = log_reg_output(X_test, y_test, theta_hat_gd, 9, 0)
y_LR.shape

In [ ]:
LR_error = evaluate_error(y_test, y_LR)
LR_error

In [ ]:
y_GDA = GDA_output(X_test, y_test, d, mu_1, mu_0, sigma, phi, 9, 0)
y_GDA.shape

In [ ]:
GDA_error = evaluate_error(y_test.reshape(-1, 1), y_GDA)
GDA_error